In [1]:
import conin
from conin.common import load_model
from conin.bayesian_network.model import DiscreteBayesianNetwork
from conin.markov_network.model import DiscreteMarkovNetwork

INFO:numexpr.utils:Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [2]:
# TODO check table preamble format + write Markov version

In [3]:
def parse_bayesian_function_table(factor, node2id):
    '''
    Function to parse the preamble and function table for a DBN factor
    '''
    # TODO check on format here
    factor_node_ids = [node2id[n] for n in factor.nodes]  # indep_0, ..., indep_{N-2}, dep?
    #table_preamble = [len(factor_node_ids)] + factor_node_ids[1:] + [factor_node_ids[0]]  # N, indep_0, ..., indep_{N-2}, dep
    table_preamble = [len(factor_node_ids)] + factor_node_ids
    table_preamble = [str(d) for d in table_preamble]
    table_values = [str(v) for k,v in factor.values.items()]
    return table_preamble, table_values
    

def write_bayesian_uai_file(model, file):
    '''
    Function to serialize the parameters of a DBN in UAI format
    Inputs:  model (conin.bayesian_network.model.DiscreteBayesianNetwork) - model to be converted
             file  (str) - filename of output (will end in .uai)
    Outputs: <file>.uai - written UAI file
    '''

    # get node2id mapping
    node2id = {n:i for i, n in enumerate(model.nodes)}

    # get domain sizes
    sizes = [str(len(model.states[n])) for n in model.nodes]

    # get factors
    factors = [cpd.to_factor() for cpd in model.cpds]

    # get function table preamble and values for each vactor
    tables = [parse_bayesian_function_table(factor, node2id) for factor in factors]

    # open file
    with open(file.split('.')[0]+'.uai', 'w') as f:
        # Preamble
        f.write('BAYES'+'\n')  # network type
        f.write(str(len(node2id))+'\n')  # num. nodes
        f.write(' '.join(sizes)+'\n')  # node domain sizes
        f.write(str(len(tables))+'\n')  # num. function tables
        for (table_preamble, _) in tables:
            f.write(' '.join(table_preamble)+'\n')  # table preamble

        # Function Tables
        for (_, table_values) in tables:
            f.write(str(len(table_values))+'\n')  # len table
            f.write(' ' + ' '.join(table_values)+'\n')  # table values

In [4]:
model = load_model('asia.bif')
print(isinstance(model, DiscreteBayesianNetwork))

True


In [5]:
write_bayesian_uai_file(model, 'asia_test')

In [6]:
with open('asia_test.uai', 'r') as f:
    lines = f.readlines()
for line in lines:
    print(line.strip())

BAYES
8
2 2 2 2 2 2 2 2
8
1 0
2 5 1
3 1 3 2
3 4 6 3
2 5 4
1 5
2 0 6
2 3 7
2
0.01 0.99
4
0.6 0.4 0.3 0.7
8
0.9 0.1 0.8 0.2 0.7 0.3 0.1 0.9
8
1.0 0.0 1.0 0.0 1.0 0.0 0.0 1.0
4
0.1 0.9 0.01 0.99
2
0.5 0.5
4
0.05 0.95 0.01 0.99
4
0.98 0.02 0.05 0.95
